# Capstone Project Data Science (mdavap)
# Import Library

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import joblib
import json

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, root_mean_squared_error

import warnings
warnings.filterwarnings('ignore')

# Import the dataset

In [2]:
df = pd.read_csv("../dataset/anime.csv")
df.head(5)

,id,title,title_english,type,source,episodes,status,airing,rating,score,rank,scored_by,popularity,favorites,members,synopsis,season,year,start,ending,studios,genres
0,1,Cowboy Bebop,Cowboy Bebop,TV,Original,26,Finished Airing,False,R - 17+ (violence & profanity),8.75,46,999712,43,85049,1935105,"Crime is timeless. By the year 2071, humanity ...",spring,1998,1998-04-03,1999-04-24,Sunrise,Action;Award Winning;Sci-Fi
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,Movie,Original,1,Finished Airing,False,R - 17+ (violence & profanity),8.38,212,221847,635,1661,390938,"Another day, another bounty—such is the life o...",NaN,2001,2001-09-01,NaN,Bones,Action;Sci-Fi
2,6,Trigun,Trigun,TV,Manga,26,Finished Airing,False,PG-13 - Teens 13 or older,8.22,371,383411,259,16535,789238,"Vash the Stampede is the man with a $$60,000,0...",spring,1998,1998-04-01,1998-09-30,Madhouse,Action;Adventure;Sci-Fi
3,7,Witch Hunter Robin,Witch Hunter Robin,TV,Original,26,Finished Airing,False,PG-13 - Teens 13 or older,7.24,3155,44884,1921,658,120993,"Though hidden away from the general public, Wi...",summer,2002,2002-07-03,2002-12-25,Sunrise,Action;Drama;Mystery;Supernatural
4,8,Bouken Ou Beet,Beet the Vandel Buster,TV,Manga,52,Finished Airing,False,PG - Children,6.93,4653,6912,5557,16,16091,It is the dark century and the people are suff...,fall,2004,2004-09-30,2005-09-29,Toei Animation,Action;Adventure;Fantasy


# Regression
- We will predict `score`

# Preprocessing

## One hot encoding for genres and studios

In [3]:
df_encoded = df.join(df['genres'].str.get_dummies(sep=';')).join(df['studios'].str.get_dummies(sep=';'))
df_encoded.head(5)

,id,title,title_english,type,source,episodes,status,airing,rating,score,rank,scored_by,popularity,favorites,members,synopsis,season,year,start,ending,studios,genres,Action,Adventure,Avant Garde,Award Winning,Boys Love,Comedy,Drama,Ecchi,Erotica,Fantasy,Girls Love,Gourmet,Hentai,Horror,Mystery,Romance,Sci-Fi,Slice of Life,...,Zero-G Room,Zexcs,Zuiyo,Zyc,aNCHOR,animate Film,asread.,cogitoworks,d00r works,domerica,drop,dwarf,evg,feel.,flag Co.,foodunited.,happyproject,helo.inc,iDRAGONS Creative Studio,l-a-unch・BOX,lxtl,maroyaka,miHoYoAnime,monofilmo,pH Studio,production doA,qmotri,soket,studio ALBLE,studio MOTHER,studio NAGURI,studio YOG,studio hb,studio2 Animation Lab,team Yamahitsuji,teamKG,trenova,ufotable,uzupiyo Animation & Digital Works,yell
0,1,Cowboy Bebop,Cowboy Bebop,TV,Original,26,Finished Airing,False,R - 17+ (violence & profanity),8.75,46,999712,43,85049,1935105,"Crime is timeless. By the year 2071, humanity ...",spring,1998,1998-04-03,1999-04-24,Sunrise,Action;Award Winning;Sci-Fi,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,Movie,Original,1,Finished Airing,False,R - 17+ (violence & profanity),8.38,212,221847,635,1661,390938,"Another day, another bounty—such is the life o...",NaN,2001,2001-09-01,NaN,Bones,Action;Sci-Fi,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,6,Trigun,Trigun,TV,Manga,26,Finished Airing,False,PG-13 - Teens 13 or older,8.22,371,383411,259,16535,789238,"Vash the Stampede is the man with a $$60,000,0...",spring,1998,1998-04-01,1998-09-30,Madhouse,Action;Adventure;Sci-Fi,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,7,Witch Hunter Robin,Witch Hunter Robin,TV,Original,26,Finished Airing,False,PG-13 - Teens 13 or older,7.24,3155,44884,1921,658,120993,"Though hidden away from the general public, Wi...",summer,2002,2002-07-03,2002-12-25,Sunrise,Action;Drama;Mystery;Supernatural,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8,Bouken Ou Beet,Beet the Vandel Buster,TV,Manga,52,Finished Airing,False,PG - Children,6.93,4653,6912,5557,16,16091,It is the dark century and the people are suff...,fall,2004,2004-09-30,2005-09-29,Toei Animation,Action;Adventure;Fantasy,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
df_encoded['airing'] = df_encoded['airing'].apply(lambda x: 1 if x else 0)
df_encoded.head(5)

,id,title,title_english,type,source,episodes,status,airing,rating,score,rank,scored_by,popularity,favorites,members,synopsis,season,year,start,ending,studios,genres,Action,Adventure,Avant Garde,Award Winning,Boys Love,Comedy,Drama,Ecchi,Erotica,Fantasy,Girls Love,Gourmet,Hentai,Horror,Mystery,Romance,Sci-Fi,Slice of Life,...,Zero-G Room,Zexcs,Zuiyo,Zyc,aNCHOR,animate Film,asread.,cogitoworks,d00r works,domerica,drop,dwarf,evg,feel.,flag Co.,foodunited.,happyproject,helo.inc,iDRAGONS Creative Studio,l-a-unch・BOX,lxtl,maroyaka,miHoYoAnime,monofilmo,pH Studio,production doA,qmotri,soket,studio ALBLE,studio MOTHER,studio NAGURI,studio YOG,studio hb,studio2 Animation Lab,team Yamahitsuji,teamKG,trenova,ufotable,uzupiyo Animation & Digital Works,yell
0,1,Cowboy Bebop,Cowboy Bebop,TV,Original,26,Finished Airing,0,R - 17+ (violence & profanity),8.75,46,999712,43,85049,1935105,"Crime is timeless. By the year 2071, humanity ...",spring,1998,1998-04-03,1999-04-24,Sunrise,Action;Award Winning;Sci-Fi,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,Movie,Original,1,Finished Airing,0,R - 17+ (violence & profanity),8.38,212,221847,635,1661,390938,"Another day, another bounty—such is the life o...",NaN,2001,2001-09-01,NaN,Bones,Action;Sci-Fi,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,6,Trigun,Trigun,TV,Manga,26,Finished Airing,0,PG-13 - Teens 13 or older,8.22,371,383411,259,16535,789238,"Vash the Stampede is the man with a $$60,000,0...",spring,1998,1998-04-01,1998-09-30,Madhouse,Action;Adventure;Sci-Fi,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,7,Witch Hunter Robin,Witch Hunter Robin,TV,Original,26,Finished Airing,0,PG-13 - Teens 13 or older,7.24,3155,44884,1921,658,120993,"Though hidden away from the general public, Wi...",summer,2002,2002-07-03,2002-12-25,Sunrise,Action;Drama;Mystery;Supernatural,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8,Bouken Ou Beet,Beet the Vandel Buster,TV,Manga,52,Finished Airing,0,PG - Children,6.93,4653,6912,5557,16,16091,It is the dark century and the people are suff...,fall,2004,2004-09-30,2005-09-29,Toei Animation,Action;Adventure;Fantasy,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Label encoding
- `status`
- `type`
- `source`
- `rating`
- `season`

In [5]:
df_encoded['status'].value_counts()

status
Finished Airing     27034
Not yet aired         544
Currently Airing      392
Name: count, dtype: int64

In [6]:
df_encoded['type'].value_counts()

type
TV            8203
Movie         4766
OVA           4157
ONA           3861
Music         3731
Special       1769
TV Special     737
CM             434
PV             231
unknown         81
Name: count, dtype: int64

In [7]:
df_encoded['source'].value_counts()

source
Original        11786
Manga            5396
Unknown          2854
Game             1413
Other            1253
Visual novel     1153
Light novel      1151
Novel             805
Web manga         606
4-koma manga      330
Picture book      269
Music             261
Mixed media       223
Book              217
Web novel         163
Card game          76
Radio              14
Name: count, dtype: int64

In [8]:
df_encoded['rating'].value_counts()

rating
PG-13 - Teens 13 or older         9984
G - All Ages                      8761
PG - Children                     4327
Rx - Hentai                       1553
R - 17+ (violence & profanity)    1551
R+ - Mild Nudity                  1197
no_rating                          597
Name: count, dtype: int64

In [9]:
df_encoded['season'].value_counts()

season
spring    1929
fall      1750
winter    1263
summer    1095
Name: count, dtype: int64

In [10]:
status_encoder = LabelEncoder()
type_encoder = LabelEncoder()
source_encoder = LabelEncoder()
rating_encoder = LabelEncoder()
season_encoder = LabelEncoder()


df_encoded['status'] = status_encoder.fit_transform(df_encoded['status'])
df_encoded['type'] = type_encoder.fit_transform(df_encoded['type'])
df_encoded['source'] = source_encoder.fit_transform(df_encoded['source'])
df_encoded['rating'] = rating_encoder.fit_transform(df_encoded['rating'])
df_encoded['season'] = rating_encoder.fit_transform(df_encoded['season'])

In [11]:
df_encoded.sample(1)

,id,title,title_english,type,source,episodes,status,airing,rating,score,rank,scored_by,popularity,favorites,members,synopsis,season,year,start,ending,studios,genres,Action,Adventure,Avant Garde,Award Winning,Boys Love,Comedy,Drama,Ecchi,Erotica,Fantasy,Girls Love,Gourmet,Hentai,Horror,Mystery,Romance,Sci-Fi,Slice of Life,...,Zero-G Room,Zexcs,Zuiyo,Zyc,aNCHOR,animate Film,asread.,cogitoworks,d00r works,domerica,drop,dwarf,evg,feel.,flag Co.,foodunited.,happyproject,helo.inc,iDRAGONS Creative Studio,l-a-unch・BOX,lxtl,maroyaka,miHoYoAnime,monofilmo,pH Studio,production doA,qmotri,soket,studio ALBLE,studio MOTHER,studio NAGURI,studio YOG,studio hb,studio2 Animation Lab,team Yamahitsuji,teamKG,trenova,ufotable,uzupiyo Animation & Digital Works,yell
20105,47392,Wangzhe? Bie Nao!,NaN,3,3,13,1,0,2,0.0,15075,0,23040,0,127,NaN,4,2017,2017-06-03,2019-01-01,NaN,Comedy;Fantasy,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Drop useless column

In [12]:
df_encoded = df_encoded.drop(['id', 'title', 'title_english', 'synopsis', 'start', 'ending', 'studios', 'genres'], axis=1) # studios and genres were hot encoded
df_encoded.sample(1)

,type,source,episodes,status,airing,rating,score,rank,scored_by,popularity,favorites,members,season,year,Action,Adventure,Avant Garde,Award Winning,Boys Love,Comedy,Drama,Ecchi,Erotica,Fantasy,Girls Love,Gourmet,Hentai,Horror,Mystery,Romance,Sci-Fi,Slice of Life,Sports,Supernatural,Suspense,100studio,10Gauge,1IN,2:10 Animation,33 Collective,...,Zero-G Room,Zexcs,Zuiyo,Zyc,aNCHOR,animate Film,asread.,cogitoworks,d00r works,domerica,drop,dwarf,evg,feel.,flag Co.,foodunited.,happyproject,helo.inc,iDRAGONS Creative Studio,l-a-unch・BOX,lxtl,maroyaka,miHoYoAnime,monofilmo,pH Studio,production doA,qmotri,soket,studio ALBLE,studio MOTHER,studio NAGURI,studio YOG,studio hb,studio2 Animation Lab,team Yamahitsuji,teamKG,trenova,ufotable,uzupiyo Animation & Digital Works,yell
4134,7,13,14,1,0,0,0.0,15825,0,16856,0,503,0,2007,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Split dataset for Training and Testing

In [13]:
X = df_encoded.drop('score', axis=1)
y = df_encoded['score']

# Split the Dataset into Train and Test with test 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# Modeling

In [14]:
model_regressions = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Decision Tree Regressor': DecisionTreeRegressor(random_state=99),
    'Support Vector Regression': SVR(),
    'Multi-layer Perceptron regressor': MLPRegressor(),
}

In [15]:
model_evaluations = {}

for model_name, model in model_regressions.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    model_evaluations[model_name] = {
        'R^2': model.score(X_test, y_test),
        'MSE': mean_squared_error(y_test, predictions),
        'RMSE': root_mean_squared_error(y_test, predictions),
    }
    print(f'{model_name} Has finished training and evaluated!')

Linear Regression Has finished training and evaluated!
Ridge Regression Has finished training and evaluated!
Lasso Regression Has finished training and evaluated!
Decision Tree Regressor Has finished training and evaluated!
Support Vector Regression Has finished training and evaluated!
Multi-layer Perceptron regressor Has finished training and evaluated!


# Evaluation

In [29]:
def PrintModelEvaluation(name, evaluation):
    R2 = evaluation['R^2']
    MSE = evaluation['MSE']
    RMSE = evaluation['RMSE']

    print('-'*20, name, '-'*20)
    print(f'R2 Score: {R2}')
    print(f'MSE Score: {MSE}')
    print(f'RMSE Score: {RMSE}')

    print('-' * (40 + len(name) + 2))

for name, evaluation in model_evaluations.items():
    PrintModelEvaluation(name, evaluation)

-------------------- Linear Regression --------------------
R2 Score: 0.7637655249119264
MSE Score: 2.3043391065524474
RMSE Score: 1.5180049757996341
-----------------------------------------------------------
-------------------- Ridge Regression --------------------
R2 Score: 0.7665999537220703
MSE Score: 2.2766907916758035
RMSE Score: 1.5088707007811515
----------------------------------------------------------
-------------------- Lasso Regression --------------------
R2 Score: 0.7020753597439016
MSE Score: 2.9060931902159797
RMSE Score: 1.7047267200979692
----------------------------------------------------------
-------------------- Decision Tree Regressor --------------------
R2 Score: 0.9964354918453283
MSE Score: 0.03476984268859492
RMSE Score: 0.18646673346362594
-----------------------------------------------------------------
-------------------- Support Vector Regression --------------------
R2 Score: 0.7297632304928676
MSE Score: 2.6360130365033383
RMSE Score: 1.623580314

# Save the best model
- We choose Decision Tree Regressor with reasons below:
    - R2 Score: 0.996 (closest to 1.0)
    - MSE Score: 0.035 (lowest error)
    - RMSE Score: 0.186 (lowest error)
- Explaination
    - The R2 score of 0.996 indicates that the model explains 99.64% of the variance in the target variable, which is excellent. All other models have R2 scores below 0.77.
    - The MSE (Mean Squared Error) of 0.035 is significantly lower than all other models, with the next best being Ridge Regression at 2.277.
    - The RMSE (Root Mean Square Error) of 0.186 is also much lower than the other models, with Ridge Regression coming in second at 1.509.

In [31]:
base_path = '../model/regression/'

In [33]:
decision_tree_regressor = model_regressions['Decision Tree Regressor']
joblib.dump(decision_tree_regressor, f'{base_path}decision_tree_regressor.joblib')
print(f'Decision Tree Regressor has been saved to disk!')

Decision Tree Regressor has been saved to disk!


## Dont forget save the label encoder and column order

In [34]:
joblib.dump(status_encoder, f'{base_path}status_encoder.joblib')
joblib.dump(type_encoder, f'{base_path}type_encoder.joblib')
joblib.dump(source_encoder, f'{base_path}source_encoder.joblib')
joblib.dump(rating_encoder, f'{base_path}rating_encoder.joblib')
joblib.dump(season_encoder, f'{base_path}season_encoder.joblib')
print(f'All label encoder has been saved to disk!')

All label encoder has been saved to disk!


In [35]:
with open(f'{base_path}columns.json', 'w', encoding='utf-8') as out:
    json.dump(list(X.columns.values), out, indent=4)

print(f'Column order has been saved to disk!')

Column order has been saved to disk!
